# Signature Provider Plugin

The Signature Provider is a crucial component of SPSDK that facilitates secure signing operations using custom hardware security module (HSM). It acts as an interface between the SPSDK and the HSM, allowing developers to leverage the cryptographic capabilities of the HSM for signing data securely.

## 1. Benefits of using the Signature Provider
 
 - **Enhanced Security**:
   - The Signature Provider enables the use of HSMs, which are designed to provide a high level of security for cryptographic operations. By keeping private keys within the HSM, the risk of key exposure is significantly reduced.
 
 - **Custom HSM Support**:
   - One of the standout features of the Signature Provider is its ability to work with custom HSMs. This flexibility allows organizations to integrate their existing security infrastructure into the SPSDK, ensuring that they can utilize their preferred hardware solutions.
 
 - **Interoperability**:
   - The Signature Provider is designed to be compatible with various HSMs and cryptographic standards. This interoperability ensures that organizations can switch between different HSMs or use multiple HSMs without significant changes to their codebase.
 
 - **Audit and Compliance**:
   - Using an HSM for signing operations can help organizations meet regulatory compliance requirements. The Signature Provider can facilitate logging and auditing of cryptographic operations, providing a clear trail for compliance purposes.


This notebook describes how to setup a Signature Provider plugin

When signing the data with SPSDK, there are two options:
- Using the local private key (not recommended)
- Using the remote signing service(HSM)

Let's look at the second option and setup Signature Provider 


In [2]:
PLUGINS_DIR = "plugins/"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

## 2. Custom HSM setup

First, we need to start the custom HSM. In order to do that, open the [HSM notebook](sahsm.ipynb) and follow the instructions there. Once you are done, come back and continue here.

Now the HSM should be up and running. In order to test the functionality of HSM from previous step, run a simple test:

In [1]:
import requests

# rsa2048 sign
response = requests.get("http://127.0.0.1:5000/signer/rsa2048/0?data=b'ABC'")
print(f"RSA2048: {response.json()}")

response = requests.get("http://127.0.0.1:5000/signer/secp384r1/0?data=b'ABC'")
print(f"SECP384R1: {response.json()}")

RSA2048: {'signature': 'QWjBWnbG7QtninaD6R9dQZqGiMZZskdVLCV1peXZEp43SJx3PATOoXTIQhvLhOZ5Q0f1683dtGAkEzb1aHKY05fIw2iPAGNHsL7IAe5nH0t3dOaCvemlodzAbb8GDpdahUHBURpnJOsgqYccZZOR6E3GSuIwD8qKBlZ7sGomtwzrBGuNHU5AG8U0J+8hLhExpEttd953mtnyMnC5aq3W30SbwU+7lZDAc2jIJn1PltVUetdHOVyGSPi4yAGZIlnzgYD8vpse2xlPP+3Ifdfuu3ckkNSZ0xzmK8adehKGTqD5hlpnP9iWPd7lio+82SovjmQ552RwwtRGbFmqC2qEkg=='}
SECP384R1: {'signature': 'ADePXaZhKL1AW7SHanQQOAyqFGIIRJzPhs9SiW2MInXD/GH9t9fZX0NNmdQ0ZL37uV+C58iLOTUY/iFtNmO9yJsRyU8ncBOUTLQbEbGCIbHoi5HUKk9KcBb0LS3tYU3X'}


## 3. Signature Provider plugin

Plugins extend the existing SPSDK functionality with additional features. 

In order to use remote signing, a Signature Provider plugin used for communication with HSM must be implemented.

Explore the `plugins\sasp.py`. It will be used later on.

### 3.1 Plugin API

Note that the SuperAwesomeSP plugin is derived from `spsdk.crypto.SignatureProvider` base class.

- The derived class has to implement:
  - `identifier: str`: class attribute that identifies the concrete implementation of SignatureProvider
  - `sign(bytes) -> bytes`: method which performs the actual signing
  - `signature_length -> str`: property which returns a length of a signature
- The derived class can also optionally implement:
  - `info() -> str`: method which returns information about the signature provider (for debugging purposes). The default implementation returns a class name as a string
  - `verify_public_key(bytes) -> bool`: method which verifies if a given public key matches a private key.
 
> Omitting the implementation of optional methods such as `info()` does not break the functionality of application.

### 3.2 Signature Formats

The signature must meet following formats:

- RSA: Standard format
- ECC: Extracted public numbers r+s stored in big endian or DER-formatted signature